In [78]:
# # this notebook is divided into total 5 parts
# Part 1:  Generating hot 100 songs --- using python library beautifulsoup
#          and using  webscraping method exctracted position song title and artist name
# Part 2:  Using spotipy librabry make connection to spotify 
# Part 3:  asking user's wish for song (constraint -- atleast it should be more than 3 letters)  
#          if user's song is in top 100 it will give random song from Hot 100
# Part 4:  if the userr song is not in top 100 than exctracting the audio feature of user's song from spotify
# Part 5:  Fianl_data_K_cluster.csv finle contains large play list with artist name and divided into 13 clusters using
#          K clustering algoritham (Fianl_data_K_cluster.csv is the output of "KMeans_Clusturing_Big_Platlist" notebook )  
#          user,s song is transformed according to final_data_K_cluster.csv
# output : recomandation to user a song which suits its wish song    
        

In [79]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from random import randint
from pickle import load
import pandas as pd
from pandas import json_normalize

# Part 1: Generating Top 100 Songs

In [80]:
url = "http://www.popvortex.com/music/charts/top-100-songs.php"
response = requests.get(url)
print("Response code: ",response.status_code)

# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")

Response code:  200


In [81]:
# Getting the Position
position = []
for i in soup.select("p.chart-position"):
    position.append(i.get_text())


# Extracting Titles    
title = []
# for t in soup.select(".title-artist"):
#     title.append(t.cite.get_text())
    
for t in soup.select("p.title-artist cite.title"):
    title.append(t.get_text())
    
    
# Extracting Artist Name    
artist_name = []
for t2 in soup.select("p.title-artist em.artist"):
    artist_name.append(t2.get_text())
        
        
repertoire = pd.DataFrame({"position":position,
                           "song_title":title,
                           "Artist_Name":artist_name                           
                          })

#Coverting the song_title into lower case -- in order to compare it easily
repertoire["song_title"] = repertoire["song_title"].str.lower()
repertoire["song_title"] = repertoire["song_title"].str.strip()        

# Part 2 (Making connection with spotify)

In [82]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up your Spotify credentials
secrets_file = open("secrets.txt","r")
string = secrets_file.read()

secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
        

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))       

# Part 3: Taking input from user 

In [83]:
user_song = input("Please give your Song Title: ")


Please give your Song Title: hr


In [84]:
if len(user_song) > 3:
    user_song = user_song.lower().strip()
else:
    user_song = input("Please give your Song Title: ")

    
result = repertoire["song_title"].apply(lambda x: user_song in x)
if (result == False).all() != True:
     print(repertoire['song_title'][randint(0,99)])
else:
     print("Sorry we could not find your song in Top 100, we would like to recommend song")   
        
#result.value_counts() # it check weather the user song is in the list or not        

Please give your Song Title: heart
Sorry we could not find your song in Top 100, we would like to recommend song


# Part 4 Finding the audio features and related cluser for user's song

In [85]:
song_name = user_song 
#Searching user's song on spotify
results = sp.search(q=song_name, limit=1)
track = results['tracks']['items'][0]
track_uri = track["uri"]
#track_uri 



# Retrieve audio features
audio_features_user_song_1 = sp.audio_features(track_uri)
#print(audio_features_user_song_1)




# converting audio feautres into dataframe
audio_features_user_song_1 = json_normalize(audio_features_user_song_1)
audio_features_user_song_1["popularity"] = track["popularity"]


# Taking only Numerical data
audio_features_user_song = audio_features_user_song_1[["popularity","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms","time_signature"]]
#audio_features_user_song 

# Part 5 Scaling and modeling user's song and finding the related cluster

In [86]:
pscaler = load(open("scaler_new.p",'rb'))
X_scaled_user_song = pscaler.transform(audio_features_user_song)

# assigning the above song into cluster
kmodel = load(open("kmeans.pkl","rb"))
user_Song_cluster = kmodel.predict(audio_features_user_song)

C:\Users\Denish\anaconda1\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


In [87]:
# Assigning the random song from cluster 

song_data = pd.read_csv("Fianl_data_K_cluster.csv")
song_data.drop(["Unnamed: 0"],axis =1,inplace =True)
for i in range(0,song_data["Num_cluster"].nunique()):
    if i == user_Song_cluster:
        new_df = song_data[song_data["Num_cluster"] == i]
        new_df.reset_index(drop=True, inplace=True)
        recomende_song = new_df['song_name'][randint(new_df.index.min(),new_df.index.max())]
print(f"Your recomended song is: {recomende_song} \nfrom artist : {new_df[new_df['song_name'] == recomende_song]['name'].iloc[0]}")        

Your recomended song is: Your Loft My Acid 
from artist : Death In Vegas
